In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
from sklearn.metrics import f1_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/week4-practice-hackathon-2024/sample_submission.csv
/kaggle/input/week4-practice-hackathon-2024/Train.csv
/kaggle/input/week4-practice-hackathon-2024/Test.csv


**Here's an outline of the project:**
  
    1.Download the dataset
    2.Some exploratory data analysis
    3.Some pre-processing
    4.Training a baseline model
    5.According to the score doing some feature engineering
    6.Train different models
    7.Fine tune the hyper parameters of the best models
    8.Submission file

# 1.Download the Dataset

 

***Loading Training Set***

In [42]:
df = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Train.csv")
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V45,V47,V50,V51,V52,V59,V63,V72,V75,class
0,0.0,1.0,243.0,29,4528.0,138.0,3.0,29,201.0,0.0,...,2,3,71.0,0,0,46.0,1,2,0,1
1,0.0,20.0,8.0,14,4183.0,183.0,4.0,26,156.0,0.0,...,5,7,20.0,0,2,13.0,9,6,0,1
2,14.0,2.0,2.0,9,1113.0,15.0,37.0,31,78.0,1.0,...,7,5,11.0,0,0,7.0,9,2,1,0
3,0.0,175.0,17.0,16,4250.0,68.0,8.0,30,152.0,0.0,...,1,3,0.0,1,1,31.0,6,4,5,1
4,2.0,-1.0,1.0,3,0.0,0.0,2.0,5,6.0,1.0,...,1,5,0.0,0,0,2.0,8,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40771,0.0,0.0,4.0,5,41928.0,1836.0,1.0,11,307.0,0.0,...,1,3,31.0,5,1,1.0,1,1,2,1
40772,1.0,90.0,6.0,5,1376.0,5.0,71.0,5,155.0,1.0,...,1,8,1.0,2,0,31.0,7,13,1,0
40773,0.0,2.0,10.0,35,1424.0,46.0,17.0,42,190.0,0.0,...,1,8,0.0,4,0,3.0,6,17,1,1
40774,1.0,22.0,1.0,1,0.0,0.0,1.0,1,8.0,1.0,...,8,5,0.0,1,0,0.0,6,3,2,0


***Loading Training Set***

In [43]:
test_df=pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Test.csv")
test_df

,Index,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V43,V45,V47,V50,V51,V52,V59,V63,V72,V75
0,40977,3.0,97.0,2.0,3,475.0,23.0,11.0,10,146.0,...,123.0,1,8,0.0,0,1,5.0,6,14,0
1,24157,7.0,29.0,11.0,10,8.0,2.0,11.0,16,19.0,...,26.0,1,8,2.0,4,1,45.0,6,2,26
2,56238,2.0,9.0,5.0,1,1003.0,29.0,2.0,29,54.0,...,10.0,1,5,35.0,1,1,6.0,6,2,3
3,55411,0.0,0.0,22.0,17,1527.0,28.0,4.0,24,28.0,...,95.0,8,8,1.0,2,1,15.0,8,1,3
4,53175,13.0,-1.0,46.0,6,4.0,5.0,16.0,15,22.0,...,30.0,5,8,13.0,1,1,5.0,6,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17471,17197,5.0,8.0,2.0,17,1025.0,114.0,5.0,17,49.0,...,1835.0,1,8,0.0,9,0,3.0,6,2,1
17472,14094,1.0,22.0,4.0,10,1.0,7.0,7.0,10,50.0,...,65.0,5,0,0.0,0,0,2.0,2,2,1
17473,34598,2.0,66.0,4.0,6,1025.0,38.0,2.0,38,38.0,...,104.0,1,8,0.0,9,0,1.0,7,1,7
17474,17845,1.0,944.0,6.0,6,1084.0,6.0,14.0,23,68.0,...,2.0,5,8,19.0,10,2,0.0,7,2,0


# 2.Exploratory Data Analysis:

***Training Set***

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40776 entries, 0 to 40775
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      40776 non-null  float64
 1   V2      40776 non-null  float64
 2   V3      40776 non-null  float64
 3   V4      40776 non-null  int64  
 4   V5      40776 non-null  float64
 5   V6      40776 non-null  float64
 6   V7      40776 non-null  float64
 7   V8      40776 non-null  int64  
 8   V9      40776 non-null  float64
 9   V10     40776 non-null  float64
 10  V11     40776 non-null  int64  
 11  V13     40776 non-null  float64
 12  V19     40776 non-null  int64  
 13  V22     40776 non-null  int64  
 14  V30     40776 non-null  int64  
 15  V33     40776 non-null  int64  
 16  V35     40776 non-null  int64  
 17  V36     40776 non-null  int64  
 18  V40     40776 non-null  float64
 19  V41     40776 non-null  int64  
 20  V42     40776 non-null  float64
 21  V43     40776 non-null  float64
 22

In [45]:
df.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V45,V47,V50,V51,V52,V59,V63,V72,V75,class
count,40776.000000,40776.000000,40776.000000,40776.000000,40776.000000,40776.000000,40776.000000,40776.000000,40776.000000,40776.000000,...,40776.000000,40776.000000,40776.000000,40776.000000,40776.000000,40776.00000,40776.000000,40776.000000,40776.00000,40776.000000
mean,4.167206,107.012041,28.035879,8.250221,1714.740092,76.849029,28.728124,17.533721,149.706837,0.736144,...,2.658598,6.238817,19.605233,3.231043,0.674563,19.08917,5.937071,3.171204,3.19482,0.501251
std,9.952973,389.279030,394.669313,8.942177,4789.147970,243.058478,83.490689,13.917054,272.661541,0.714021,...,2.680439,2.586094,71.480746,6.009488,0.710048,64.79848,2.014287,5.747578,5.77309,0.500005
min,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,1.000000,3.000000,2.000000,15.000000,4.000000,3.000000,6.000000,18.000000,0.000000,...,1.000000,5.000000,1.000000,1.000000,0.000000,1.00000,6.000000,1.000000,1.00000,0.000000
50%,1.000000,5.000000,7.000000,5.000000,401.000000,18.000000,8.000000,14.000000,59.000000,1.000000,...,1.000000,8.000000,4.000000,1.000000,1.000000,4.00000,6.000000,1.000000,1.00000,1.000000
75%,4.000000,42.000000,18.000000,11.000000,1658.000000,62.000000,23.000000,27.000000,166.000000,1.000000,...,5.000000,8.000000,14.000000,3.000000,1.000000,14.00000,7.000000,3.000000,3.00000,1.000000
max,816.000000,12274.000000,32743.000000,130.000000,373199.000000,14843.000000,4245.000000,52.000000,7082.000000,7.000000,...,8.000000,10.000000,4552.000000,136.000000,7.000000,2863.00000,13.000000,117.000000,136.00000,1.000000


***Test Set***

In [46]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17476 entries, 0 to 17475
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Index   17476 non-null  int64  
 1   V1      17476 non-null  float64
 2   V2      17476 non-null  float64
 3   V3      17476 non-null  float64
 4   V4      17476 non-null  int64  
 5   V5      17476 non-null  float64
 6   V6      17476 non-null  float64
 7   V7      17476 non-null  float64
 8   V8      17476 non-null  int64  
 9   V9      17476 non-null  float64
 10  V10     17476 non-null  float64
 11  V11     17476 non-null  int64  
 12  V13     17476 non-null  float64
 13  V19     17476 non-null  int64  
 14  V22     17476 non-null  int64  
 15  V30     17476 non-null  int64  
 16  V33     17476 non-null  int64  
 17  V35     17476 non-null  int64  
 18  V36     17476 non-null  int64  
 19  V40     17476 non-null  float64
 20  V41     17476 non-null  int64  
 21  V42     17476 non-null  float64
 22

In [47]:
test_df.describe()

,Index,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V43,V45,V47,V50,V51,V52,V59,V63,V72,V75
count,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,...,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000,17476.000000
mean,29098.451019,4.225223,107.446155,30.881323,8.337778,1698.186141,73.085946,29.132811,17.567178,147.737411,...,95.599336,2.650721,6.269856,19.480659,3.176814,0.681392,19.730201,5.924582,3.161192,3.184768
std,16794.335033,10.009722,387.326722,446.483656,9.085835,4118.557429,205.779485,99.316646,13.985736,291.249131,...,265.494886,2.661117,2.582070,68.281644,5.736272,0.714132,70.073521,2.017642,5.615862,5.904322
min,4.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,14627.000000,0.000000,1.000000,3.000000,2.000000,15.000000,4.000000,3.000000,6.000000,17.000000,...,6.000000,1.000000,5.000000,1.000000,1.000000,0.000000,1.000000,6.000000,1.000000,1.000000
50%,28951.500000,1.000000,5.000000,7.000000,5.000000,410.500000,18.000000,8.000000,14.000000,58.000000,...,25.000000,1.000000,8.000000,4.000000,1.000000,1.000000,4.000000,6.000000,1.000000,1.000000
75%,43596.250000,4.000000,42.000000,19.000000,11.000000,1668.000000,61.000000,23.000000,28.000000,163.000000,...,84.000000,5.000000,8.000000,15.000000,3.000000,1.000000,14.000000,7.000000,3.000000,3.000000
max,58250.000000,287.000000,11797.000000,31391.000000,102.000000,155475.000000,5941.000000,6605.000000,53.000000,12768.000000,...,8828.000000,8.000000,10.000000,2635.000000,109.000000,6.000000,3270.000000,13.000000,90.000000,107.000000


**Observations:**
   * Column data types are mixed,we should make them all either float or int
   * From description we can see data is centred around median and outliers are affecting the     dataset badly

# 3.Prepare Dataset for Training:

   *  Split Training and Validation Set
   *  Fill/Remove missing values or discrepancies
   *  Extract Inputs & Outputs
                
            

# Split Training & Validation Set

 We'll set aside 20% of the training data as the validation set,to evaluate the models we train on previously unseen data.
 
 Since there is no time factor,we can pick a random 20% fraction

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [50]:
len(train_df), len(val_df)

(32620, 8156)

# Fill/Remove the Missing Values
  
  There are no missing values in our sample , but if there were,we could drop that rows or in the time of pre-processing we can also impute some values there.

In [51]:
train_df = train_df.dropna()
val_df = val_df.dropna()

# Extract Inputs and Outputs

In [52]:
train_df.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V13', 'V19', 'V22', 'V30', 'V33', 'V35', 'V36', 'V40', 'V41', 'V42',
       'V43', 'V45', 'V47', 'V50', 'V51', 'V52', 'V59', 'V63', 'V72', 'V75',
       'class'],
      dtype='object')

In [53]:
input_cols = train_df.columns[0:-1]

In [54]:
target_cols = train_df.columns[-1]

# Training

In [55]:
train_inputs = train_df[input_cols]

In [56]:
train_targets = train_df[target_cols]

In [57]:
train_inputs 

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V43,V45,V47,V50,V51,V52,V59,V63,V72,V75
1513,0.0,11.0,1.0,1,1447.0,78.0,10.0,38,246.0,0.0,...,24.0,1,5,0.0,4,1,6.0,6,3,6
1118,7.0,101.0,7.0,7,4.0,5.0,137.0,22,92.0,3.0,...,11.0,8,5,12.0,0,2,0.0,7,3,4
21406,0.0,0.0,3.0,5,2247.0,7.0,1.0,4,5.0,0.0,...,1.0,1,3,17.0,0,0,10.0,8,1,2
16591,4.0,28.0,4.0,5,1.0,5.0,9.0,25,31.0,1.0,...,36.0,2,8,5.0,0,1,1.0,6,21,4
12854,1.0,10.0,1.0,2,0.0,0.0,1.0,6,12.0,1.0,...,50.0,1,5,14.0,0,1,0.0,6,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1.0,-1.0,380.0,12,139.0,35.0,4.0,33,899.0,1.0,...,0.0,1,8,1.0,1,1,8.0,6,3,6
11284,0.0,0.0,7.0,8,17320.0,642.0,6.0,23,368.0,0.0,...,970.0,1,5,0.0,4,1,6.0,7,25,7
38158,0.0,0.0,2.0,6,2966.0,70.0,25.0,10,70.0,0.0,...,2.0,8,5,11.0,0,2,62.0,8,5,0
860,0.0,1.0,74.0,14,2020.0,107.0,4.0,22,38.0,0.0,...,0.0,8,8,26.0,4,0,79.0,6,0,0


In [58]:
train_targets

1513     0
1118     1
21406    1
16591    0
12854    0
        ..
6265     0
11284    1
38158    1
860      0
15795    0
Name: class, Length: 32620, dtype: int64

# Validation

In [59]:
val_inputs = val_df[input_cols]

In [60]:
val_targets = val_df[target_cols]

In [61]:
val_inputs 

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V43,V45,V47,V50,V51,V52,V59,V63,V72,V75
430,1.0,15.0,3.0,2,47.0,1.0,9.0,26,239.0,1.0,...,4.0,7,6,25.0,2,1,7.0,6,4,8
14032,0.0,150.0,26.0,3,2387.0,105.0,12.0,17,47.0,0.0,...,53.0,0,5,1.0,9,1,25.0,6,0,1
35478,6.0,6.0,7.0,1,620.0,67.0,6.0,2,2.0,1.0,...,26.0,1,3,2.0,0,2,0.0,4,3,1
11192,16.0,232.0,2.0,19,766.0,23.0,16.0,28,33.0,1.0,...,66.0,0,3,9.0,5,1,0.0,8,0,1
31264,1.0,0.0,13.0,6,28.0,12.0,1.0,7,10.0,1.0,...,5.0,1,5,8.0,2,0,1.0,6,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23498,1.0,6.0,1.0,5,0.0,1.0,12.0,3,10.0,1.0,...,0.0,0,5,38.0,1,1,4.0,7,0,0
8305,1.0,2.0,2.0,4,105.0,4.0,1.0,12,12.0,1.0,...,110.0,1,5,3.0,3,1,0.0,6,1,2
5144,2.0,45.0,12.0,0,384.0,4.0,3.0,0,47.0,1.0,...,2.0,0,8,7.0,5,1,0.0,6,1,1
8450,0.0,0.0,1.0,1,7400.0,15.0,3.0,6,15.0,0.0,...,72.0,1,10,13.0,0,0,4.0,7,3,1


In [62]:
val_targets

430      0
14032    1
35478    0
11192    1
31264    0
        ..
23498    0
8305     0
5144     1
8450     0
26530    1
Name: class, Length: 8156, dtype: int64

# Test

In [63]:
test_inputs = test_df[input_cols]

In [64]:
test_inputs

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V43,V45,V47,V50,V51,V52,V59,V63,V72,V75
0,3.0,97.0,2.0,3,475.0,23.0,11.0,10,146.0,1.0,...,123.0,1,8,0.0,0,1,5.0,6,14,0
1,7.0,29.0,11.0,10,8.0,2.0,11.0,16,19.0,2.0,...,26.0,1,8,2.0,4,1,45.0,6,2,26
2,2.0,9.0,5.0,1,1003.0,29.0,2.0,29,54.0,1.0,...,10.0,1,5,35.0,1,1,6.0,6,2,3
3,0.0,0.0,22.0,17,1527.0,28.0,4.0,24,28.0,0.0,...,95.0,8,8,1.0,2,1,15.0,8,1,3
4,13.0,-1.0,46.0,6,4.0,5.0,16.0,15,22.0,1.0,...,30.0,5,8,13.0,1,1,5.0,6,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17471,5.0,8.0,2.0,17,1025.0,114.0,5.0,17,49.0,1.0,...,1835.0,1,8,0.0,9,0,3.0,6,2,1
17472,1.0,22.0,4.0,10,1.0,7.0,7.0,10,50.0,1.0,...,65.0,5,0,0.0,0,0,2.0,2,2,1
17473,2.0,66.0,4.0,6,1025.0,38.0,2.0,38,38.0,1.0,...,104.0,1,8,0.0,9,0,1.0,7,1,7
17474,1.0,944.0,6.0,6,1084.0,6.0,14.0,23,68.0,1.0,...,2.0,5,8,19.0,10,2,0.0,7,2,0


# 4.Train Hardcoded & Baseline Models
  

In [65]:
model=LogisticRegression(max_iter=500)

In [66]:
model.fit(train_inputs,train_targets)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

In [67]:
train_preds = model.predict(train_inputs)

In [68]:
train_preds

array([0, 0, 1, ..., 1, 1, 0])

In [69]:
val_preds = model.predict(val_inputs)

In [70]:
val_preds

array([0, 1, 0, ..., 0, 1, 0])

In [71]:
def f1score(targets,preds):
    return f1_score(targets,preds)

In [72]:
train_f1 = f1score(train_targets,train_preds)
train_f1

0.6416063362415692

In [73]:
val_f1 = f1score(val_targets,val_preds)
val_f1

0.6423891129032258

# 6.Feature Engineering

Here tha data is purely numerical,as for now no feature engineering needed

# 7. Data Pre-processing

# Scaling & One-Hot Encoding

 * This data has outliers,and the data is mostly centred around median,suitable scaling method needed
 * Here simply no categorical column,no one hot encoding needed

In [74]:
from sklearn.preprocessing import RobustScaler

In [75]:
scaler=RobustScaler()

In [77]:
train_inputs_scale = scaler.fit_transform(train_inputs)
val_inputs_scale = scaler.fit_transform(val_inputs)

# 8.Train & Evaluate Different Models

We'll train each of the following & submit predictions to kaggle:
     
  * Logistic Regression
  * Random forest
  * Gradient Boosting

In [85]:
def evaluate(model):
    train_preds = model.predict(train_inputs)
    train_f1 = f1score(train_targets,train_preds)
    val_preds = model.predict(val_inputs)
    val_f1 = f1score(val_targets,val_preds)
    return train_f1, val_f1, train_preds, val_preds

# Logistic Regression

In [91]:
model1 = LogisticRegression(random_state=42,n_jobs=-1,C=0.001, class_weight=None, max_iter=500, penalty='elasticnet',l1_ratio=0.40, solver='saga', tol=0.000324521152615054)

In [92]:
model1.fit(train_inputs_scale,train_targets)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.001, l1_ratio=0.4, max_iter=500, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga',
                   tol=0.000324521152615054)

In [93]:
evaluate(model1)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


(0.6942364050797786,
 0.6884351180954174,
 array([1, 0, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]))

# Random Forest

In [94]:
from sklearn.ensemble import RandomForestClassifier

In [120]:
model2 = RandomForestClassifier(random_state=42,n_jobs=-1,n_estimators=1200,max_depth=6,max_leaf_nodes=64,max_features='sqrt',min_samples_split=5,min_samples_leaf=2)

In [121]:
%%time
model2.fit(train_inputs_scale,train_targets)

CPU times: user 58.4 s, sys: 384 ms, total: 58.8 s
Wall time: 16 s


RandomForestClassifier(max_depth=6, max_leaf_nodes=64, min_samples_leaf=2,
                       min_samples_split=5, n_estimators=1200, n_jobs=-1,
                       random_state=42)

In [122]:
evaluate(model2)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


(0.6870885514795523,
 0.6778265150836988,
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 0, 1, 1]))

# Gradient Boosting

In [109]:
from xgboost import XGBClassifier

In [165]:
model3=XGBClassifier(random_state=42,n_jobs=-1,colsample_bytree=0.5, gamma=0, learning_rate=0.01, max_depth=3, min_child_weight=3, n_estimators=500, reg_alpha=0.5, reg_lambda=10, subsample=0.7, objective="binary:logistic")

In [166]:
model3.fit(train_inputs_scale,train_targets)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

In [167]:
evaluate(model3)

(0.6877820086721106,
 0.6794940317120969,
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 0, 1, 1]))

# 3 Model Ensembling

In [173]:
from sklearn.ensemble import VotingClassifier

# Assuming model1, model2, model3 are already defined and trained
voting_clf = VotingClassifier(estimators=[('model1', model1), ('model2', model2), ('model3', model3)], voting='soft')
voting_clf.fit(train_inputs_scale,train_targets)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


VotingClassifier(estimators=[('model1',
                              LogisticRegression(C=0.001, l1_ratio=0.4,
                                                 max_iter=500, n_jobs=-1,
                                                 penalty='elasticnet',
                                                 random_state=42, solver='saga',
                                                 tol=0.000324521152615054)),
                             ('model2',
                              RandomForestClassifier(max_depth=6,
                                                     max_leaf_nodes=64,
                                                     min_samples_leaf=2,
                                                     min_samples_split=5,
                                                     n_estimators=1200,
                                                     n_jobs=-1,
                                                     random_state=42)),
                             ('model3',
                              XGBClassifie...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None, min_child_weight=3,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=500, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [174]:
evaluate(voting_clf)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


(0.6933321129467759,
 0.6862962962962963,
 array([1, 0, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [175]:
from sklearn.ensemble import StackingClassifier

# Assuming model1, model2, model3 are already defined and trained
estimators = [('model1', model1), ('model2', model2), ('model3', model3)]
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(),cv=5)
stacking_clf.fit(train_inputs_scale,train_targets)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

StackingClassifier(cv=5,
                   estimators=[('model1',
                                LogisticRegression(C=0.001, l1_ratio=0.4,
                                                   max_iter=500, n_jobs=-1,
                                                   penalty='elasticnet',
                                                   random_state=42,
                                                   solver='saga',
                                                   tol=0.000324521152615054)),
                               ('model2',
                                RandomForestClassifier(max_depth=6,
                                                       max_leaf_nodes=64,
                                                       min_samples_leaf=2,
                                                       min_samples_split=5,
                                                       n_estimators=1200,
                                                       n_jobs=-1,
                                                       random_state=42)),
                               ('model3',
                                XGBCl...
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=0.01, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=3,
                                              max_leaves=None,
                                              min_child_weight=3, missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=500, n_jobs=-1,
                                              num_parallel_tree=None,
                                              random_state=42, ...))],
                   final_estimator=LogisticRegression())

In [176]:
evaluate(stacking_clf)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


(0.6889962710443284,
 0.6808206958073149,
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 0, 1, 1]))

In [177]:
test_df = pd.read_csv("/kaggle/input/week4-practice-hackathon-2024/Test.csv")
test_df

,Index,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V43,V45,V47,V50,V51,V52,V59,V63,V72,V75
0,40977,3.0,97.0,2.0,3,475.0,23.0,11.0,10,146.0,...,123.0,1,8,0.0,0,1,5.0,6,14,0
1,24157,7.0,29.0,11.0,10,8.0,2.0,11.0,16,19.0,...,26.0,1,8,2.0,4,1,45.0,6,2,26
2,56238,2.0,9.0,5.0,1,1003.0,29.0,2.0,29,54.0,...,10.0,1,5,35.0,1,1,6.0,6,2,3
3,55411,0.0,0.0,22.0,17,1527.0,28.0,4.0,24,28.0,...,95.0,8,8,1.0,2,1,15.0,8,1,3
4,53175,13.0,-1.0,46.0,6,4.0,5.0,16.0,15,22.0,...,30.0,5,8,13.0,1,1,5.0,6,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17471,17197,5.0,8.0,2.0,17,1025.0,114.0,5.0,17,49.0,...,1835.0,1,8,0.0,9,0,3.0,6,2,1
17472,14094,1.0,22.0,4.0,10,1.0,7.0,7.0,10,50.0,...,65.0,5,0,0.0,0,0,2.0,2,2,1
17473,34598,2.0,66.0,4.0,6,1025.0,38.0,2.0,38,38.0,...,104.0,1,8,0.0,9,0,1.0,7,1,7
17474,17845,1.0,944.0,6.0,6,1084.0,6.0,14.0,23,68.0,...,2.0,5,8,19.0,10,2,0.0,7,2,0


In [179]:
Index=test_df['Index']
test_df.drop(['Index'],axis=1,inplace=True)

In [180]:
y_test = voting_clf.predict(test_df)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [181]:
y_test

array([1, 0, 1, ..., 1, 1, 0])

In [182]:
result = pd.DataFrame({
    'Index': Index,
    'class': y_test
})

In [183]:
result

,Index,class
0,40977,1
1,24157,0
2,56238,1
3,55411,1
4,53175,1
...,...,...
17471,17197,1
17472,14094,1
17473,34598,1
17474,17845,1


In [185]:
result.to_csv("submission.csv", index=False) #(for making submission file)